In [1]:
# search in the hyperparameter space with W&B sweep
import logging
from ast import literal_eval as load
import pandas as pd
import numpy as np
from ast import literal_eval
import sklearn

from simpletransformers.classification import (
    ClassificationArgs,
    ClassificationModel,
)

In [2]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

df = pd.read_csv('triples.csv')
df.insert(loc=0, column='idx', value=np.arange(len(df)))
sent_num=len(df)

In [3]:
def convert(arr):
    trip_list=[]
    ls=[]
    for i in range(len(arr)):
        #if arr[i,3]=='[]' or ((arr[i,3]!='[]') and (load(arr[i,3])[0][1][0]>load(arr[i,4])[0][1][0])):
        if arr[i,4]!='[]':
            if arr[i,3]=='[]' or ((arr[i,3]!='[]') and (load(arr[i,3])[0][1][0]>load(arr[i,4])[0][1][0])):
                np = load(arr[i,4])[0]
                uni_name=arr[i,1]
                uni_name=(uni_name[0].upper()+uni_name[1:]).replace('-',' ')
                triple=[uni_name,'has',np[0]]
                trip_list.append(triple)
                word_ls = arr[i,2].split(' ')
                word_ls.insert(np[1][0], '[[')
                word_ls.insert(np[1][1]+1, ']]')
                unit = arr[i,1]
                unit = (unit[0].upper()+unit[1:]).replace('-',' ')
                unit_ls = ['[[']+(unit.split(' '))+[']]']
                word_ls = unit_ls+[':']+word_ls
                flg=0
                if arr[i,8]=='[]':
                    trip_ls = []
                else:
                    trip_ls = load(arr[i,8])
                    for trip in trip_ls:
                        if trip[1]=='has' and trip[2]==np[0]:
                            flg=1
                            break
                ls.append([int(arr[i, 0]), unit, np[0], trip_ls, ' '.join(word_ls), flg])
        #else:
            #missed+=len(load(arr[i,8]))
    dataframe = pd.DataFrame(ls)
    dataframe.columns = ['idx','info_unit', 'np', 'triples', 'text', 'labels']
    return dataframe,trip_list

In [4]:
df=df.values
df,trip_list=convert(df)

In [5]:
model_args = ClassificationArgs()

model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.manual_seed = 1
model_args.fp16 = False
model_args.use_multiprocessing = True
model_args.do_lower_case = True  # when using uncased model

In [6]:
def triple_F1(ref, pred):
    return 0

In [7]:
# Create a TransformerModel
model = ClassificationModel(
    "bert",
    "../rel/outputsD/best_model",
    args=model_args,
)
result, model_outputs, wrong_predictions = model.eval_model(df, F1_score=triple_F1)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


  0%|          | 0/1421 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/178 [00:00<?, ?it/s]

/opt/conda/envs/hl/lib/python3.9/site-packages/sklearn/metrics/_classification.py:870: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
INFO:simpletransformers.classification.classification_model:{'mcc': 0.0, 'tp': 0, 'tn': 632, 'fp': 789, 'fn': 0, 'F1_score': 0, 'eval_loss': 1.677608103773902}


In [8]:
preds = list(model_outputs.argmax(axis=1))
df['preds']=preds
df['cand']=trip_list
df.loc[df['preds']==0,'cand']=None
data=[]
for i in range(sent_num):
    temp = list(df[df['idx']==i]['cand'])
    temp = [t for t in temp if t]
    data.append(str(temp))

In [9]:
data

['[]',
 '[]',
 "[['Model', 'has', 'RNNGs']]",
 '[]',
 "[['Model', 'has', 'discriminative model']]",
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 "[['Model', 'has', 'bi-directional transformer architecture']]",
 '[]',
 '[]',
 '[]',
 "[['Experimental setup', 'has', 'CNN models']]",
 "[['Experimental setup', 'has', 'learning rate']]",
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 "[['Results', 'has', 'Our CNN base model']]",
 "[['Results', 'has', 'Named Entity Recognition']]",
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 "[['Results', 'has', 'beam size']]",
 '[]',
 '[]',
 "[['Results', 'has', 'Seq2seq approach']]",
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 "[['Hyperparameters', 'has', 'embedding layer']]",
 '[]',
 '[]',
 '[]',
 '[]',
 "[['Results', 'has', 'LSTM + A model']]",
 "[['Results', 'has', 'difference']]",
 "[['Results', 'has', 'LSTM + A']]",
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 "[['Ablation analysis', 'has', 'RG']]",
 '[]',
 '[]',
 '[]',
 '[]',


In [10]:
data=pd.DataFrame(data,columns=['triple_D'])
data.to_csv('D.csv',index=False)